In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import time

# Try to import yahooquery for fast filtering
try:
    from yahooquery import Ticker
except ImportError:
    print("Please install yahooquery: pip install yahooquery")

# ==========================================
# 1. STEP 1: FETCH UNIVERSE
# ==========================================
def get_raw_universe():
    print("--- STEP 1: Fetching Raw Stock Lists ---")
    tickers = []
    try:
        url = "http://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
        df_us = pd.read_csv(url, sep='|')
        df_us = df_us[(df_us['Test Issue'] == 'N') & (df_us['ETF'] == 'N')]
        us_list = df_us['Symbol'].str.replace('.', '-', regex=False).dropna().unique().tolist()
        us_list = [t for t in us_list if len(t) < 5 and '$' not in t]
        tickers.extend(us_list)
        print(f"   -> Found {len(us_list)} US candidates.")
    except Exception as e:
        print(f"   Error fetching US list: {e}")
    return list(set(tickers))

# ==========================================
# 2. STEP 2: FINANCIALS ONLY FILTER
# ==========================================
def filter_financials_universe(ticker_list):
    print(f"\n--- STEP 2: Filtering for Financial Services (YahooQuery) ---")
    
    MIN_PRICE = 5.0
    MIN_CAP_US = 300_000_000
    
    valid_candidates = []
    chunk_size = 500
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Filtering batch {i} - {min(i+chunk_size, len(ticker_list))}...", end='\r')
        
        try:
            yq = Ticker(chunk, asynchronous=False)
            data = yq.get_modules("summaryProfile summaryDetail price financialData")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): continue
                
                # Sector Check
                profile = data[symbol].get('summaryProfile', {})
                sector = profile.get('sector', 'Unknown')
                if 'Financial' not in sector and 'Real Estate' not in sector: continue

                # Basic Data
                price_mod = data[symbol].get('price', {})
                curr_price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                
                fin_mod = data[symbol].get('financialData', {})
                rec_key = fin_mod.get('recommendationKey', 'none')
                if rec_key and isinstance(rec_key, str): rec_key = rec_key.lower().strip()
                
                # Filters
                if curr_price < MIN_PRICE: continue
                if mkt_cap < MIN_CAP_US: continue
                if rec_key not in ['buy', 'strong_buy', 'strong buy']: continue

                valid_candidates.append({
                    'Ticker': symbol, 
                    'Price': curr_price,
                    'Sector': sector,
                    'Rating': rec_key
                })
        except:
            continue
            
    df = pd.DataFrame(valid_candidates)
    print(f"\n   -> Filter complete. Survivors: {len(df)}")
    return df

# ==========================================
# 3. DATA ENRICHMENT (Get P/E, P/B, Yield)
# ==========================================
def fetch_financial_data(df):
    """
    Fetches the deep financial data for the list of survivors.
    Returns the 'Master' DataFrame.
    """
    if df is None or df.empty:
        print("No financial stocks to analyze.")
        return None

    print(f"\n--- STEP 3: FETCHING METRICS FOR {len(df)} STOCKS ---")
    bank_data = []
    
    for index, row in df.iterrows():
        ticker = row['Ticker']
        if index % 5 == 0: print(f"   Fetching {index}/{len(df)}...", end='\r')
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            pe = info.get('trailingPE', np.nan)
            pb = info.get('priceToBook', np.nan)
            roe = info.get('returnOnEquity', np.nan)
            div_yield = info.get('dividendYield', 0)
            if div_yield is None: div_yield = 0
            
            recom = info.get('recommendationMean', None)
            
            bank_data.append({
                'Ticker': ticker,
                'Price': row['Price'],
                'P/E': pe,
                'P/B': pb,
                'ROE': roe,
                'Yield%': round(div_yield * 100, 2),
                'Recom': float(recom) if recom is not None else 3.0,
                'Sector': row.get('Sector', 'Financial')
            })
        except: continue
            
    master_df = pd.DataFrame(bank_data)
    
    # Force numeric types
    cols = ['P/E', 'P/B', 'ROE', 'Yield%', 'Recom']
    for col in cols: master_df[col] = pd.to_numeric(master_df[col], errors='coerce')
    
    return master_df

# ==========================================
# 4. VIEW: VALUE PICKS (Matches Screenshot 1)
# ==========================================
def get_value_picks(df):
    """
    Returns a dataframe filtered for VALUE and sorted by P/B.
    """
    # Filter: P/E < 15, P/B < 1.2, ROE > 8%
    mask = (df['P/E'] < 15) & (df['P/B'] < 1.2) & (df['ROE'] > 0.08)
    value_df = df[mask].copy()
    
    # Sort: Cheapest Assets (P/B) first
    value_df = value_df.sort_values(by='P/B', ascending=True)
    
    # Columns matching your screenshot
    cols = ['Ticker', 'Price', 'P/B', 'P/E', 'ROE', 'Yield%', 'Recom']
    return value_df[cols]

# ==========================================
# 5. VIEW: INCOME PICKS (Matches Screenshot 2)
# ==========================================
def get_income_picks(df):
    """
    Returns a dataframe filtered for INCOME and sorted by Yield.
    """
    # Filter: Yield > 2.5%, P/E < 20, Buy Rating
    mask = (df['Yield%'] >= 2.5) & (df['P/E'] < 20) & (df['Recom'] <= 2.5)
    income_df = df[mask].copy()
    
    # Sort: Highest Yield first
    income_df = income_df.sort_values(by='Yield%', ascending=False)
    
    # Columns matching your screenshot
    cols = ['Ticker', 'Price', 'Yield%', 'P/E', 'Recom', 'Sector']
    return income_df[cols]

# ==========================================
# EXECUTION BLOCK
# ==========================================
raw_tickers = get_raw_universe()
financial_df = filter_financials_universe(raw_tickers)

if not financial_df.empty:
    # 1. Get the Big List
    master_df = fetch_financial_data(financial_df)
    
    if master_df is not None and not master_df.empty:
        # 2. Call the specific functions you wanted
        df_value = get_value_picks(master_df)
        df_income = get_income_picks(master_df)
        
        print("\n\n✅ Done! Two DataFrames are ready for Data Wrangler:")
        print("   1. df_value  (Undervalued)")
        print("   2. df_income (High Dividend)")
        
        # Display small preview
        print("\n--- Value Preview ---")
        print(df_value.head())

--- STEP 1: Fetching Raw Stock Lists ---
   -> Found 5974 US candidates.

--- STEP 2: Filtering for Financial Services (YahooQuery) ---
   Filtering batch 5500 - 5974...
   -> Filter complete. Survivors: 385

--- STEP 3: FETCHING METRICS FOR 385 STOCKS ---
   Fetching 380/385...

✅ Done! Two DataFrames are ready for Data Wrangler:
   1. df_value  (Undervalued)
   2. df_income (High Dividend)

--- Value Preview ---
    Ticker  Price       P/B        P/E      ROE  Yield%    Recom
278    MFG   7.32  0.257894  14.076924  0.09280   268.0  2.50000
378   MARA   9.59  0.703853   3.731518  0.23066     0.0  1.92308
166   BMNR  28.31  0.764391   2.114264  0.08016     4.0  1.00000
355   BCSF  13.93  0.800713   9.673611  0.08185  1206.0  2.00000
257    OTF  14.13  0.818229   7.850000  0.10580   991.0  2.00000


In [2]:
import os
import time

# ==========================================
# 4. SEPARATE SAVE FUNCTION (Financials)
# ==========================================
def save_financials_to_excel(df):
    """
    Saves the 'final_financial_results' to OneDrive.
    """
    if df is None or df.empty:
        print("❌ No data found. Did you run the main analysis cell first?")
        return

    # 1. Setup Filename
    today_date = time.strftime("%Y-%m-%d")
    file_nickname = f"Financial_Stock_Picks_{today_date}.xlsx"
    
    # 2. Setup Folder (Explicit Path)
    onedrive_folder = r"C:\Users\James\OneDrive - McMaster University\YFinance Stock Picks"
    full_path = os.path.join(onedrive_folder, file_nickname)

    print(f"Saving to: {full_path}...")

    # 3. Filter the Lists again for separate sheets
    value_picks = df[df['Strategy'] == 'Value (Undervalued)'].sort_values(by='P/B', ascending=True)
    income_picks = df[df['Strategy'] == 'Income (Dividend)'].sort_values(by='Yield%', ascending=False)

    try:
        if not os.path.exists(onedrive_folder):
            os.makedirs(onedrive_folder, exist_ok=True)

        with pd.ExcelWriter(full_path, engine='openpyxl') as writer:
            # The 'df' already contains the 'P/E' column, so it will be saved automatically.
            value_picks.to_excel(writer, sheet_name='Value Picks', index=False)
            income_picks.to_excel(writer, sheet_name='Income Picks', index=False)
            df.to_excel(writer, sheet_name='All Financials', index=False)
            
        print(f"✅ Success! Excel file created.")
        print(f"   - Value Picks: {len(value_picks)}")
        print(f"   - Income Picks: {len(income_picks)}")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Close the Excel file if it is currently open.")

# --- TRIGGER SAVE ---
if 'final_financial_results' in locals():
    save_financials_to_excel(final_financial_results)
else:
    print("⚠️ 'final_financial_results' is missing. Please run the analysis code block first.")

Saving to: C:\Users\James\OneDrive - McMaster University\YFinance Stock Picks\Financial_Stock_Picks_2025-12-28.xlsx...
✅ Success! Excel file created.
   - Value Picks: 2
   - Income Picks: 142
